In [2]:
import json, os

data_folder = "/Users/user/Courses/Semantic Parsing/datasets/wikiwiki"

outf = open(os.path.join(data_folder, "train.1k.seqs"), 'w')

missing_count = 0
total_count = 0
line_count = 0

with open(os.path.join(data_folder, "train.1k.jsonl")) as inf:
    for line in inf:
        obj = json.loads(line.strip())
        context = obj["context"]
        edict = {}
        for e in obj["mention2entity"]:
            if obj["mention2entity"][e][0] not in edict:
                edict[obj["mention2entity"][e][0]] = []
            edict[obj["mention2entity"][e][0]].append(e)
        for e in edict:
            edict[e].sort(key = lambda x: len(x), reverse=True)
            if e in obj["entity2type"]:
                edict[e] = (edict[e][0], obj["entity2type"][e][0])
            else:
                edict[e] = (edict[e][0], "missing")
        
        sentences = obj["context"].strip().split('. ')
        for s in sentences:
            sout = s.strip()
            sets = []
            for e in edict:
                if edict[e][0] in sout:
                    sets.append(edict[e])
            if len(sets) > 0:
                if "missing" not in [i[1] for i in sets]:
                    outf.write(json.dumps({"sequence": sout, "spans": sets}) + '\n')
                    total_count += 1
                else:
                    missing_count += 1
        line_count += 1

        if line_count % 100000 == 0:
            print('Processed:', line_count)
            print('Num sentences:', total_count)
            print('Missing count:', missing_count)
            print()
outf.close()

In [8]:
import json
import re

data_folder = "/Users/user/Courses/Semantic Parsing/datasets/wikiwiki"

outf = open(os.path.join(data_folder, "train.1k.proc.seqs"), 'w')

with open(os.path.join(data_folder, "train.1k.seqs")) as inf:
    j = 0
    for line in inf:
        sobj = json.loads(line.strip())
        text = sobj["sequence"]
        otext = text
        spans = sobj["spans"]
        spans.sort(key = lambda x: len(x[0]), reverse=True)
        for s in spans:
            all_ind = [(0, 0)] + [(m.start(), m.end()) for m in re.finditer(s[0], text)] + [(len(text), len(text))]
            new_text = ""
            for i in range(1, len(all_ind) - 1):
                new_text += text[all_ind[i-1][1] : all_ind[i][0]]
                new_text += ' [' + s[1] + ' ' + text[all_ind[i][0]: all_ind[i][1]] + ' ' + s[1] + '] '
            new_text += text[all_ind[-2][1] : all_ind[-1][0]]
            text = ' '.join([a.strip() for a in new_text.split(' ') if a.strip() != ''])
#         print(otext)
#         print(spans)
#         print(text)
#         print()
#         j += 1
#         if j > 20:
#             break
        outf.write(text + '\n')
            
outf.close()

In [10]:
from transformers import AutoTokenizer
counts = []
max_tar = 0

tokenizer = AutoTokenizer.from_pretrained('roberta-base')

outf = open(os.path.join(data_folder, "train.1k.proc.tsv"), 'w')

with open(os.path.join(data_folder, "train.1k.proc.seqs")) as inf:
    for line in inf:
        words = line.strip().split()
        utt = ""
        utt_toks = []
        tar = []
        text_so_far = []
        count = 0

        for w in words:
            if '[Q' in w or (w[0] == 'Q' and w[-1] == ']'):
                text_to_tok = ' '.join(text_so_far)
                if count > 0:
                    text_to_tok = ' ' + text_to_tok
                if len(text_so_far) > 0:
                    utt += text_to_tok
                    toks = tokenizer.tokenize(text_to_tok)
                    for t in toks:
                        utt_toks.append(t)
                        tar.append('@ptr_' + str(count))
                        count += 1
                    text_so_far = []
                tar.append(w)
            else:
                text_so_far.append(w)
            
        if len(text_so_far) > 0:
            text_to_tok = ' ' + ' '.join(text_so_far)
            utt += text_to_tok
            toks = tokenizer.tokenize(text_to_tok)
            for t in toks:
                utt_toks.append(t)
                tar.append('@ptr_' + str(count))
                count += 1

        tar_length = len(tar)
        if tar_length > max_tar:
            max_tar = tar_length
        tar = ' '.join(tar)
        outf.write(utt + '\t' + ' '.join(utt_toks) + '\t' + tar + '\n')
        counts.append(count)
            
print(max(counts))
print(max_tar)

245
285
